In [2]:
import open3d as o3d
import numpy as np
import os
import matplotlib.pyplot as plt
from functools import reduce

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def recenter_model(model):
    mean_cov = model.compute_mean_and_covariance()
    mean = mean_cov[0]
    covariance_matrix = mean_cov[1]

    eigen_values, eigen_vectors = np.linalg.eigh(covariance_matrix)

    # 特征向量与特征值关联，并按特征值排序（特征值最大的对应主要的方向）
    eig_index = np.argsort(eigen_values)[::-1]
    principal_axes = eigen_vectors[:, eig_index]
    # principal_axes = principal_axes.transpose()

    # the first axis of pca should be y axis for a human
    principal_axes[:,[0,1]] = principal_axes[:,[1,0]]

    if principal_axes[:, 0] [0]< 0:
        principal_axes[:,0]*= -1
    if principal_axes[:,1][1] < 0:
        principal_axes[:,1]*= -1
    if principal_axes[:,2][2] < 0:
        principal_axes[:,2]*= -1
    # print(principal_axes)

    # new_z_axis = np.array([principal_axes[:,2][0], 0, principal_axes[:,2][1]])

    # z_axis = np.array([0, 0, 1])

    # rotation_axis = np.cross(z_axis, new_z_axis)
    # # print(rotation_axis)

    # # 计算旋转角度（dot product和arccos）
    # cos_theta = np.dot(z_axis, new_z_axis) / (np.linalg.norm(z_axis) * np.linalg.norm(new_z_axis))
    # angle = np.arccos(cos_theta)

    # # 计算旋转矩阵（Rodrigues' rotation formula）
    # K = np.array([
    #     [0, -rotation_axis[2], rotation_axis[1]],
    #     [rotation_axis[2], 0, -rotation_axis[0]],
    #     [-rotation_axis[1], rotation_axis[0], 0]
    # ])
    # identity_matrix = np.eye(3)
    # rotation_matrix = identity_matrix + np.sin(-angle) * K + (1 - np.cos(-angle)) * (K @ K)

    # model.rotate(rotation_matrix, center=mean)
    model.rotate(principal_axes.transpose(), center=mean)

def read_point_cloud(filename:str):
    pcd = o3d.io.read_point_cloud(filename)
    return pcd


def render(geometries):
    o3d.visualization.draw_geometries(geometries)
    
def flip(matrix):
    result = np.copy(matrix)
    result[1:3,:] = -1 * result[1:3,:]
    return result

In [4]:
# files = list_all_files()
models = [ '/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply',
          '/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0501/pcd_0.ply']
intrinsic =  o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.Kinect2ColorCameraDefault)
class AnyObject:
    pass
args = AnyObject() 
args.mask_input_path = '/home/demoy/3dProjection/cdcl_output'
args.depth_input_path = '/home/demoy/3dProjection/depths'
args.extrinsic_input_path = '/home/demoy/3dProjection/extrinsics'
args.confidence_input_path = '/home/demoy/3dProjection/segs'
args.num_views = 10
args.k = 100
args.radius = 0.05
args.smooth = 100
args.output_path = '/home/demoy/3dProjection/confidence_search'

#part_ids = [ 0,  2,  4,  5,  6,  8, 13, 14, 16, 17, 18, 19, 20, 21, 24]
# the order is: (left right flipped)
# background, head, torso, left upper arm ,right upper arm, left forearm, right forearm,
#  left hand, right hand, left thigh, right thigh, left shank, right shank, left foot, right foot
part_ids = [0, 13, 2, 5, 8, 19, 20, 4, 24, 18, 6, 21, 16, 14, 17]

r_chan = np.array([0, 127, 255, 255, 255, 127, 255, 127, 0, 0, 0, 0, 127, 255, 255])
g_chan = np.array( [0, 127, 0, 127, 255, 0, 0, 127, 255, 0, 255, 127, 255, 127, 255])
b_chan = np.array( [0, 127, 0, 0, 0, 255, 255, 0, 255, 255, 0, 255, 127, 127, 127])

In [5]:
# intrinsic.width/height 
intrinsic =  o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.Kinect2ColorCameraDefault)
print(intrinsic.height)
print(intrinsic.width)

1080
1920


In [6]:
# load point cloud and reset all points' colors to black
original_pcd = o3d.io.read_point_cloud("/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply")
points = np.asarray(original_pcd.points)
points -= points.mean(0)
original_pcd.transform(np.array([[1,0,0,0], 
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1]]))
recenter_model(original_pcd)
original_pcd.normals = o3d.utility.Vector3dVector([])

# pcd_color = (np.tile([0,0,0],(np.asarray(original_pcd.colors).shape[0],1)))
# original_pcd.colors = o3d.utility.Vector3dVector(pcd_color)
render([original_pcd])

# calculate average distance between points in the point cloud
t = original_pcd.compute_nearest_neighbor_distance()
print('average distance between points in the point cloud = ',np.array(t).mean())
print(original_pcd.points)

average distance between points in the point cloud =  0.0015608837553829575
std::vector<Eigen::Vector3d> with 799951 elements.
Use numpy.asarray() to access data.


In [16]:
# mark all visiable points

# reproject from depth, find the nearest point and mark it as visiable
id = r"/home/demoy/3dProjection/THuman2.0 Samples-20231208T170459Z-001/THuman2.0 Samples/0500/pcd_0.ply".split(os.sep)[-2]

valid_pcds = []
for i in range(10):
    view_path = os.path.join(args.mask_input_path, f"{id}_image{i+1}_cdcl_segmask.npy")
    
    depth_path = os.path.join(args.depth_input_path, f"{id}_depth{i+1}.npy")
    extrinsic_path = os.path.join(args.extrinsic_input_path, f"{id}_extrinsic{i+1}.npy")
    
    depth = np.load(depth_path).astype(np.float32)
    view = np.load(view_path).astype(np.uint8)
    extrinsic = np.load(extrinsic_path).astype(np.float32)
    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    o3d.geometry.Image(view), o3d.geometry.Image(depth), convert_rgb_to_intensity=False, depth_scale=1)

    pcd = o3d.geometry.PointCloud.create_from_depth_image(o3d.geometry.Image(depth), intrinsic, flip(extrinsic), depth_scale=1)
    
    pcd.remove_non_finite_points()
    # render([pcd])

    dists = original_pcd.compute_point_cloud_distance(pcd)
    dists = np.asarray(dists)
    ind = np.where(dists < 0.005)[0]
    new_pcd = original_pcd.select_by_index(ind)
    valid_pcds.append(new_pcd)
    

valid_pcd = reduce(lambda x,y:x+y, valid_pcds)

valid_pcd.remove_duplicated_points()
render([valid_pcd])
print(valid_pcd.points)
# assert(len(pcds) == args.num_views)

# Pcd_merged = (reduce(lambda x, y: x + y, pcds))

std::vector<Eigen::Vector3d> with 738535 elements.
Use numpy.asarray() to access data.


In [15]:
# visualize the points that are labelled as unseen

original_points = np.asarray(original_pcd.points)
valid_points = np.asarray(valid_pcd.points)

original_points_set = set([tuple(original_point) for original_point in original_points])
valid_points_set = set([tuple(valid_point) for valid_point in valid_points])
invalid_points = np.array([x for x in original_points_set - valid_points_set])
invalid_pcd = o3d.geometry.PointCloud()
invalid_pcd.points = o3d.utility.Vector3dVector(invalid_points)
render([invalid_pcd])